In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json


%matplotlib inline 

In [2]:
# Load .env enviroment variables
load_dotenv() 

True

In [3]:
# Set current amount of crypto assets
my_btc = 1.2 
my_eth = 5.3

In [4]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"  
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD" 

In [5]:
# Fetch current BTC price
fetch_btc_price = requests.get(btc_url) 

# Fetch current ETH price
fetch_eth_price = requests.get(eth_url)

# Compute current value of my crpto

current_eth_price_json = json.loads(fetch_eth_price.content)
current_btc_price_json = json.loads(fetch_btc_price.content)

current_eth_price = current_eth_price_json['data']['1027']['quotes']['USD']['price']
current_btc_price = current_btc_price_json['data']['1']['quotes']['USD']['price']

my_eth_value = current_eth_price * my_eth
my_btc_value = current_btc_price * my_btc

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}") 

# Get crypto total
total_crypto = my_eth_value  +  my_btc_value

The current value of your 1.2 BTC is $59514.00
The current value of your 5.3 ETH is $11622.26


## Collect Investments Data Using Alpaca: SPY (stocks) and AGG (bonds)

In [6]:
# Current amount of shares
my_spy_shares = 50
my_agg_shares = 200

In [7]:
# Set Alpaca API key and secret

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY") 

In [8]:
 # Create the Alpaca API object

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2") 

In [9]:
 # Format current date as ISO format
    
today = pd.Timestamp("2021-04-23", tz="America/New_York").isoformat() 

In [10]:
# Set the tickers

tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Create and empty DataFrame for closing prices

df_closing_prices = pd.DataFrame()


# Get current closing prices for SPY and AGG

df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    end = today
).df

df_closing_prices["AGG"] = df_portfolio["AGG"]["close"]
df_closing_prices["SPY"] = df_portfolio["SPY"]["close"]


# Drop the time component of the date

df_closing_prices.index = df_closing_prices.index.date


# preview dataframe

display(df_closing_prices.head())


# Pick AGG and SPY close prices

agg_close_price = df_portfolio["AGG"]["close"][0]
spy_close_price = df_portfolio["SPY"]["close"][0]


# Print AGG and SPY close prices

print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}") 



,AGG,SPY
2021-04-23,114.75,416.72


Current AGG closing price: $114.75
Current SPY closing price: $416.72


In [13]:
# Compute the current value of shares

my_spy_value = my_spy_shares * spy_close_price
my_agg_value = my_agg_shares * agg_close_price


# Print current The current value of both shares

print(f"The current value of your {my_agg_shares} AGG shares is ${my_agg_value:0.2f}") 
print (f"The current value of your {my_spy_shares} SPY shares is ${my_spy_value:0.2f}")

# Get shares total
total_shares = my_spy_value + my_agg_value

print(total_shares)


The current value of your 200 AGG shares is $22950.00
The current value of your 50 SPY shares is $20836.00
43786.0


## Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = 12000
savings = {"Amount":[total_crypto,total_shares]}


# Create savings DataFrame
df_savings = pd.DataFrame(savings, index=["crypto","shares"])

# Display savings DataFrame
display(df_savings) 

In [ ]:
# Plot savings pie chart
df_savings.plot(kind='pie',y="Amount",startangle=90)

In [ ]:
 # Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_savings = total_crypto + total_shares

# Validate saving health
if  total_savings > emergency_fund:
    print("Congratulations! You have enough money in your emergency fund")
elif total_savings == emergency_fund:
    print("Hooray, you reachgd your financial goals")  
else :
    dollars_left_to_reach_goal =  emergency_fund - total_savings
    print(f"You need ${dollars_left_to_reach_goal} to reach your goal")

 
    ## Monte Carlo Simulation


In [ ]:
# Set start and end dates of five years back from today.
start_date = pd.Timestamp('2016-04-23', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-04-23', tz='America/New_York').isoformat() 

In [ ]:
# Set timeframe to '1D'
timeframe = "1D"

# Set the ticker information
tickers = ["SPY","AGG"]

 # Get 5 years' worth of historical data for SPY and AGG
df_ticker = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# Display sample data
df_ticker.head() 

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns starting with one year 
MC_oneyear = MCSimulation(
    portfolio_data = df_ticker,
    weights = [.40,.60],
    num_simulation = 100,
    num_trading_days = 252
) 


MC_thirtyyear = MCSimulation(
    portfolio_data = df_ticker,
    weights = [.40,.60],
    num_simulation = 500,
    num_trading_days = 252*30
) 

# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_thirtyyear.calc_cumulative_return() 

